In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_yield']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 

id_o = data.Crop_type[data.Crop_type=='Others'].index.tolist()
id_others =  list(set(id_nona).intersection(set(id_o)))
x_others = X.iloc[id_others, :]
y_others = y [id_others]


# 标准化，神经网络一定要标准化
scaler = StandardScaler()
scaler.fit(X_sel)

X_sel = scaler.transform(X_sel)
y_sel = y_sel.values

x_rice = scaler.transform(x_rice)
y_rice = y_rice.values

x_wheat = scaler.transform(x_wheat)
y_wheat = y_wheat.values
                  
x_maize = scaler.transform(x_maize)
y_maize = y_maize.values

# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = scaler.transform(X_rice)

# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=2, n_estimators=300) # best parameters


for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_yield.txt')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [2]:
%reset -f

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_nue']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 

id_o = data.Crop_type[data.Crop_type=='Others'].index.tolist()
id_others =  list(set(id_nona).intersection(set(id_o)))
x_others = X.iloc[id_others, :]
y_others = y [id_others]


# 标准化，神经网络一定要标准化
scaler = StandardScaler()
scaler.fit(X_sel)

X_sel = scaler.transform(X_sel)
y_sel = y_sel.values

x_rice = scaler.transform(x_rice)
y_rice = y_rice.values

x_wheat = scaler.transform(x_wheat)
y_wheat = y_wheat.values
                  
x_maize = scaler.transform(x_maize)
y_maize = y_maize.values

# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = scaler.transform(X_rice)


# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=2, n_estimators=50) # best parameters

for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_nue.txt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [4]:
%reset -f

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_ch4']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 

id_o = data.Crop_type[data.Crop_type=='Others'].index.tolist()
id_others =  list(set(id_nona).intersection(set(id_o)))
x_others = X.iloc[id_others, :]
y_others = y [id_others]


# 标准化，神经网络一定要标准化
scaler = StandardScaler()
scaler.fit(X_sel)

X_sel = scaler.transform(X_sel)
y_sel = y_sel.values

x_rice = scaler.transform(x_rice)
y_rice = y_rice.values

x_wheat = scaler.transform(x_wheat)
y_wheat = y_wheat.values
                  
x_maize = scaler.transform(x_maize)
y_maize = y_maize.values
               
x_others = scaler.transform(x_others)
y_others = y_others.values


# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = scaler.transform(X_rice)


# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=2, n_estimators=150) # best parameters


for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_ch4.txt')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
%reset -f

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_n2o']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 

id_o = data.Crop_type[data.Crop_type=='Others'].index.tolist()
id_others =  list(set(id_nona).intersection(set(id_o)))
x_others = X.iloc[id_others, :]
y_others = y [id_others]


# 标准化，神经网络一定要标准化
scaler = StandardScaler()
scaler.fit(X_sel)

X_sel = scaler.transform(X_sel)
y_sel = y_sel.values

x_rice = scaler.transform(x_rice)
y_rice = y_rice.values

x_wheat = scaler.transform(x_wheat)
y_wheat = y_wheat.values
                  
x_maize = scaler.transform(x_maize)
y_maize = y_maize.values
               
x_others = scaler.transform(x_others)
y_others = y_others.values


# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = scaler.transform(X_rice)


# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=4, n_estimators=50) # best parameters


for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_n2o.txt')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [8]:
%reset -f

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_nh3']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 




# 标准化，神经网络一定要标准化
scaler = StandardScaler()
scaler.fit(X_sel)

X_sel = scaler.transform(X_sel)
y_sel = y_sel.values

x_rice = scaler.transform(x_rice)
y_rice = y_rice.values

x_wheat = scaler.transform(x_wheat)
y_wheat = y_wheat.values
                  
x_maize = scaler.transform(x_maize)
y_maize = y_maize.values



# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = scaler.transform(X_rice)


# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=4, n_estimators=50) # best parameters


for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_nh3.txt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
%reset -f

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_leaching']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 




# 标准化，神经网络一定要标准化
scaler = StandardScaler()
scaler.fit(X_sel)

X_sel = scaler.transform(X_sel)
y_sel = y_sel.values

x_rice = scaler.transform(x_rice)
y_rice = y_rice.values

x_wheat = scaler.transform(x_wheat)
y_wheat = y_wheat.values
                  
x_maize = scaler.transform(x_maize)
y_maize = y_maize.values

# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = scaler.transform(X_rice)


# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=6, n_estimators=50) # best parameters


for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_leaching.txt')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
%reset -f

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

# 读取数据
data = pd.read_csv('data_machine_learning_new.csv',index_col='ID') 
id_X = np.hstack((3, np.arange(8,99)))
id_Y = np.arange(99,113)
X = data.iloc[:,id_X]
Y = data.iloc[:,id_Y]


# 构造建模y数据，并剔除NA值
y = Y['effect_runoff']
id_nona = y[y.isnull().values==False].index.tolist()
#id_crop = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
#id_sel = list(set(id_nona).intersection(set(id_crop)))
id_sel = id_nona 

X_sel = X.iloc[id_sel, :]
y_sel = y [id_sel]

id_r = data.Crop_type[data.Crop_type=='Paddy rice'].index.tolist()
id_rice =  list(set(id_nona).intersection(set(id_r)))
x_rice = X.iloc[id_rice, :]
y_rice = y [id_rice]                
              
id_w = data.Crop_type[data.Crop_type=='Wheat'].index.tolist()
id_wheat =  list(set(id_nona).intersection(set(id_w)))
x_wheat = X.iloc[id_wheat, :]
y_wheat = y [id_wheat]  
                 
id_m = data.Crop_type[data.Crop_type=='Maize'].index.tolist()
id_maize =  list(set(id_nona).intersection(set(id_m)))
x_maize = X.iloc[id_maize, :]
y_maize = y [id_maize]                 

x_rice = x_rice.values
x_wheat = x_wheat.values
x_maize = x_maize.values


# load predictors for background
index_X = X.columns.tolist()
#读取数据rice
data_rice1 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_N_hwsd.txt', index_col = "OBJECTID")
data_rice1 = data_rice1.iloc[:,np.hstack((2, 3, 4, 8, 9, 10, np.arange(14,44)))]
data_rice2 = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/double_rice_climate.txt', index_col = "OBJECTID")
data_rice2 =  data_rice2.iloc[:,np.hstack(np.arange(2,60))]
data_rice = pd.merge(data_rice1,data_rice2,how='inner',on='OBJECTID')
index = pd.read_csv('D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/data_summary/index.csv', index_col = "ID")
data_rice.columns = index['index']
data_rice["Soil_temp_5"] = pd.to_numeric(data_rice["Soil_temp_5"],errors='coerce')
data_rice["Soil_temp_15"] = pd.to_numeric(data_rice["Soil_temp_15"],errors='coerce')
data_rice_dropna  = data_rice.replace([np.inf, -np.inf], np.nan).dropna(axis=0,how='any')
data_rice_dropna = data_rice_dropna[~((data_rice_dropna['aspect'] == 0) & 
                                        (data_rice_dropna['elevation'] == 0) & 
                                        (data_rice_dropna['hillshade'] == 0) & 
                                        (data_rice_dropna['slope'] == 0)) ]
X_rice = data_rice_dropna.iloc[:,np.arange(2,94)]
X_rice = X_rice[index_X]
X_rice_nor = X_rice.values


# bootstrap sampling 1000 times
y_rice_unknow_total=[]

ts=time.time()

model_rice = RandomForestRegressor(max_depth=8, n_estimators=100) # best parameters


for i in range(0,1000):
    print(i)
    x_train_rice, x_test_rice,  y_train_rice, y_test_rice = train_test_split(x_rice, y_rice, test_size = 0.2)
    
    model_rice.fit(x_train_rice, y_train_rice)
    
    y_rice_unknow = model_rice.predict(X_rice_nor)
    
    y_rice_unknow_total.append(y_rice_unknow)
    
    rice_lower = np.percentile(y_rice_unknow_total, 2.5, axis=0)

    rice_upper = np.percentile(y_rice_unknow_total, 97.5, axis=0)
    
    rice_sd = np.std(y_rice_unknow_total, axis=0)
    
    rice_mean = np.mean(y_rice_unknow_total, axis=0)

te = time.time()
print(te-ts)


# rice
rice_result = pd.DataFrame(columns=['X', 'Y'])
rice_result['X'] = data_rice_dropna['X']
rice_result['Y'] = data_rice_dropna['Y']
rice_result['lower'] = rice_lower
rice_result['upper'] = rice_upper
rice_result['sd'] = rice_sd
rice_result['mean'] = rice_mean
rice_result['CV'] = rice_result['sd']/rice_result['mean']
rice_result['upper_lower'] = rice_result['upper'] - rice_result['lower']
rice_result


rice_result.to_csv(r'D:/1 博后期间/papers/CRU_meta-analysis/global_analysis/modeling_mapping/uncertainty/uncertainty_double_rice_runoff.txt')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27